- To populate the Final Report(Google Sheet) using python

In [1]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread as gs
import time
import gspread_dataframe as gd

In [2]:
scope =["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',
        "https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]


In [3]:
creds=ServiceAccountCredentials.from_json_keyfile_name(r"C:\Users\MAHESH DETHE\Downloads\GROUP PROJECT API KEY.json",scope)

In [4]:
client = gs.authorize(creds)

In [5]:
# accessing google sheets
file1 = client.open("BSE500").sheet1
file2 = client.open("Income&Expense").sheet1
file3 = client.open("Final Report").sheet1

In [6]:
BSE = gd.get_as_dataframe(file1)
IncExp = gd.get_as_dataframe(file2)
Report = gd.get_as_dataframe(file3)

Make a new column in Gsheet 1 named “Delta” and populate it with (52 Week High - price)/(52 week High)

In [7]:
BSE.columns

Index(['Company', 'BSE code', 'NSE code', 'ISIN', 'Sector', 'Industry', 'Date',
       'Price', '1 day change(%)', '52 Week Low', '52 Week High', '3 Year Low',
       '3 Year High', '5 Year Low', '5 Year High', 'All Time Low',
       'All Time High', 'Market Cap(Cr)', 'Enterprise Value(Cr)',
       '1-Week Return', '1-Month Return', '3-Month Return(%)',
       '1-Year Return(%)', '3-Year Return', '5-Year Return(%)',
       '10-Year Return(%)', 'Price to Earnings', 'Median P/E', 'Price to Book',
       'Median P/B', 'Earning Yield(%)', 'Price earnings to growth',
       'Dividend Yield(%)', 'EV / EBITDA', 'Price / Sales',
       'Price / Cash Flow', 'Earning Per Share', 'Book Value Per Share',
       'Cash Flow Per Share', 'Free Cash Flow Per Share', 'Dividend Per Share',
       'Delta'],
      dtype='object')

In [8]:
Delta = (BSE['52 Week High']- BSE['Price'])/BSE['52 Week High']

In [9]:
BSE["Delta"] = Delta

In [10]:
BSE[BSE["Delta"] >0].head()

,Company,BSE code,NSE code,ISIN,Sector,Industry,Date,Price,1 day change(%),52 Week Low,...,Dividend Yield(%),EV / EBITDA,Price / Sales,Price / Cash Flow,Earning Per Share,Book Value Per Share,Cash Flow Per Share,Free Cash Flow Per Share,Dividend Per Share,Delta
0,Sanofi India Ltd.,500674.0,SANOFI,INE058A01010,Healthcare,Drugs & Pharma,2022-09-05,6158.70,0.593726,6086.30,...,7.9533,16.682293,4.980927,25.392209,425.0862,631.6506,242.956522,459.217391,490.0,0.279073
1,Page Industries Ltd.,532827.0,PAGEIND,INE761H01022,Textiles,Readymade Garments,2022-09-05,49786.20,-0.259937,31500.00,...,0.7433,51.785329,11.748089,169.843663,656.8292,1161.6259,293.088578,191.031917,370.0,0.033278
2,Procter & Gamble Hygiene & Health Care Ltd.,500459.0,PGHH,INE179A01014,Consumer Staples,Household & Personal Products,2022-09-05,14125.65,0.235943,12801.00,...,1.1318,52.711801,12.100589,53.168596,177.3681,227.2194,265.902649,270.468269,315.0,0.142153
3,Abbott India Ltd.,500488.0,ABBOTINDIA,INE358A01014,Healthcare,Drugs & Pharma,2022-09-05,18209.00,-1.264759,15514.00,...,1.5100,30.982268,7.730778,40.836169,380.5207,1402.9963,445.957647,446.903529,275.0,0.239214
4,Bosch Ltd.,500530.0,BOSCHLTD,INE323A01026,Automobile,Auto Ancillaries,2022-09-05,17591.10,0.130919,12932.45,...,1.1937,25.043204,4.027383,192.657100,438.1080,3734.3284,91.288136,46.677966,210.0,0.086177


Finding top 5 companies for each Higher,Lower,Moderate Risks

In [11]:
df_HRT = BSE[(BSE["Delta"]>0) & (BSE['Market Cap(Cr)']<4000) & 
    (BSE['10-Year Return(%)']<8)].sort_values(by = ['Dividend Per Share'],ascending=False).head()
# Top 5 High Risk Companies
top_HRT = df_HRT["Company"].to_list()
top_HRT

['MOIL Ltd.',
 'Responsive Industries Ltd.',
 'Vakrangee Ltd.',
 'Hathway Cable & Datacom Ltd.',
 'Strides Pharma Science Ltd.']

In [12]:
df_RT = BSE[  (BSE["Delta"]>0) & 
              ((BSE['Market Cap(Cr)']>4000) & (BSE['Market Cap(Cr)']<8000)) & 
              ((BSE['10-Year Return(%)']>=8) & (BSE['10-Year Return(%)']<15))].sort_values(by = ['Dividend Per Share'],
                                                                              ascending=False).head()
# Top 5 Risk Taking Companies
top_RT = df_RT["Company"].to_list()
top_RT

['The Great Eastern Shipping Company Ltd.',
 'Welspun Corp Ltd.',
 'Jyothy Labs Ltd.',
 'Gujarat State Fertilizers & Chemicals Ltd.',
 'NCC Ltd.']

In [13]:
df_MRT = BSE[  (BSE["Delta"]>0) & 
              ((BSE['Market Cap(Cr)']>8000) & (BSE['Market Cap(Cr)']<15000)) & 
              ((BSE['10-Year Return(%)']>=15) & (BSE['10-Year Return(%)']<20))].sort_values(by = ['Dividend Per Share'],
                                                                              ascending=False).head()
# Top 5 Moderate Risk Companies
top_MRT = df_MRT["Company"].to_list()
top_MRT

['Cyient Ltd.',
 'Chambal Fertilisers & Chemicals Ltd.',
 'Redington India Ltd.',
 'BASF India Ltd.',
 'Zydus Wellness Ltd.']

In [14]:
df_LRT = BSE[(BSE["Delta"]>0) & (BSE['Market Cap(Cr)']>15000) & 
    (BSE['10-Year Return(%)']>=20)].sort_values(by = ['Dividend Per Share'],ascending=False).head()
# Top 5 Low Risk Companies
top_LRT = df_LRT["Company"].to_list()
top_LRT

['Page Industries Ltd.',
 'Abbott India Ltd.',
 'MRF Ltd.',
 'Bajaj Holdings & Investment Ltd.',
 'Honeywell Automation India Ltd.']

In [15]:
df1 = BSE.loc[BSE["3-Year Return"]<0,["Industry","Company"]].groupby("Industry").count().reset_index()
df2 = BSE.loc[BSE["3-Year Return"]>0,["Industry","Company"]].groupby("Industry").count().reset_index()
table1 = df1.merge(df2,how = "inner",on = "Industry")
# table1.to_csv('_3_year_return')

In [16]:
list_allow = ['Food','Other','Transportation','Social Life','HouseHold','Apparel',
         'Education','Salary','Allowance','Beauty','Gift','Petty cash']

In [17]:
def repetation():
    for i in range(len(list_allow)):
        x = IncExp.loc[IncExp['Category'] == list_allow[i],"Account.1"].sum()
        file3.update_cell(3+i,3,x)     

Changing dropdown - IF else condition 

In [18]:
def change():
    # updating net income,expense and amount
    Net_inc = IncExp.loc[IncExp['Income/Expense'] == "Income","Account.1"].sum().astype(float)
    Net_exp = IncExp.loc[IncExp['Income/Expense'] == "Expense","Account.1"].sum().astype(float)
    file3.update_cell(16,3,Net_inc)
    file3.update_cell(17,3,Net_exp)
    Invest_Rs = Net_inc - Net_exp
    ava_invest = float(Invest_Rs)
    file3.update_cell(18,3,Invest_Rs)
    repetation()
    m = file3.cell(10,6)
    if m.value == "Moderate Risk Taking":
        for i in range(0,5):
            file3.update_cell(4+i,6,top_MRT[i-1])
            file3.update_cell(4+i,7,ava_invest/5)
    elif m.value == "High Risk Taking":
        for i in range(0,5):
            file3.update_cell(4+i,6,top_HRT[i-1])
            file3.update_cell(4+i,7,ava_invest/5)
    elif m.value == "Low Risk Taking":
        for i in range(0,5):
            file3.update_cell(4+i,6,top_LRT[i-1])
            file3.update_cell(4+i,7,ava_invest/5) 
    elif m.value == "Risk Taking" :
        for i in range(0,5):
            file3.update_cell(4+i,6,top_RT[i-1])
            file3.update_cell(4+i,7,ava_invest/5)           

In [19]:
while (True):
    change()
    time.sleep(1)